In [1]:
from llmtosql.model import WikiSQLModel
from llmtosql.trainer import Trainer
from llmtosql.dataloader import WikiSQLDataset
from llmtosql.utils.utils import plot_history, load_model, load_history
from matplotlib import pyplot as plt
import sys
import os
import json
import numpy as np
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
sys.path.insert(0, '..')

In [2]:
model = WikiSQLModel(base_model_type='bert-base-cased', attention_type='cross', col_drop=False)
# try:
#     model = load_model(model, 'model_output/model.pth')
# except Exception as e:
#     print(f'No model loaded - {e}')

2023-03-25 14:34:11 [info     ] Using cross attention mechanism


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
train_set =  WikiSQLDataset(type='dev', model=model)

2023-03-25 14:34:14 [info     ] Tokenizing dataset.


100%|██████████| 8421/8421 [00:07<00:00, 1099.45it/s]


In [4]:
train_set.maxcondsLength

4

In [5]:
# val_set = WikiSQLDataset(type='dev')

In [6]:
len(train_set.data)

8421

In [ ]:
# datasets = (train_set, val_set)

In [ ]:
config = {
    'seed': 32,
    'scheduler': None,
    'optimizer': 'adam',
    'momentum': 0.9,
    'weight_decay': 0.0,
    'lr': 0.0005,
    'criterion': 'custom',
    'metric': 'accuracy',
    'pred_function': 'softmax',
    'model_dir': 'model_output',
    'backend': 'smddp'
}

In [ ]:
trainer = Trainer(model, datasets=train_set,
                  epochs=10, batch_size=32,
                  is_parallel=False,
                  **config)

In [ ]:
trainer.fit()

In [ ]:
trainer.save_history_('model_output')

In [ ]:
history = load_history('model_output')

In [ ]:
plot_history(history)

In [ ]:
model = WikiSQLModel(base_model_type='bert-base-uncased', attention_type='sqlnet', col_drop=True)
model = load_model(model, 'model_output/model.pth')
test_loss, test_accuracy = trainer.test(model, trainer.val_loader)

In [ ]:
print(f'Model accuracy on test: {test_accuracy}')